<p style="background:#83739E; color:white; font-size: 2.1em; text-align:center"> 
    <br><b>Risk Evaluation for Retail Banks</b><br>
    <br>Presentation Notebook<br><br> 
</p>

<p style="text-align: right;">Module 4: Machine Learning<br>
Sprint 4: Machine Learning Capstone Project<br>
Author : Renato Mariano</p>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from utils import utils
from utils import EDA
from feature_models import feat_eng
from feature_models import pipelines
from feature_models import model_select


from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils.class_weight import compute_class_weight
from scipy.stats import randint, uniform

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


<h1 style="color:#774CAD">Introduction </h1>

Welcome to the Capstone Project of of the Machine Learning Module! In this sprint, we embark on an exciting journey to develop a **risk evaluation service for retail banks**, leveraging the power of data science and machine learning.

You and your friend are launching a startup focused on providing risk evaluation as a service for retail banks. This proof-of-concept (POC) plan outlines the steps you will take to investigate, analyze, and build a solution using machine learning. The dataset for analysis is obtained from Home Credit Group.

The following entity diagram shows the encompassed dataframes. They will be explored in more details during in the next sections of this notebook.

<img src="images/00_entity-relationship_diagram.PNG" width=850>

<h1 style="color:#774CAD">Exploratory Data Analysis</h1>

In order to analyze the necessity of this model, we first need to evaluate **what proportion of money is given to defaulters**. Let's start with de distribution of this defaulters and then jump into the credit amounts given to them.

Our defaulters are expressed as our **TARGET column**. As explained in the metadata, **this features shows if a client has presented difficulties to pay their loans** or not.

<img src="images/01_defaulters_proportion.png" width=400>

As we can see, our target (Loan Defaulters) variable is very **unevenly distributed** - about 92% of the client have payed their loans without any delay.

<img src="images/02_credit-amount.png" width=400>

In examining the dataset, it becomes evident that approximately **$184 billion was disbursed in credit**. Notably, **around $13 billion, equivalent to approximately 7.5% of the total credit extended, was allocated to clients who eventually defaulted**. This observation underscores the **necessity for an automated system** capable of identifying defaulters accurately. Traditional methods employed by the bank seem insufficient to capture these nuances effectively, emphasizing the need for more sophisticated approaches in credit risk assessment.

Throughout the exploration phase, numerous insights were found, prompting the creation of new features for our models. The initial features exhibited **limited correlation with the target variable**, various correlation methods were tailored to different feature types. Additionally, we conducted an analysis of numerical feature **skewness** and diligently searched for **anomalies** within the dataframe.

<h1 style="color:#774CAD">Feature Engineering</h1>

During this phase, we implemented several alterations to the dataset, including the creation of new features and the removal of anomalies. The impact of these changes on our predictive capabilities was assessed using a **LightGBM model**. To address the class imbalance inherent in our target variable, we have applied **weights to balance the classes**, adjusting primarily the **max_depth** parameter during experimentation.

Given the imbalanced nature of our classes, we choose to evaluate our model's performance using the **ROC AUC metric**. ROC AUC (Receiver Operating Characteristic Area Under the Curve) is particularly well-suited for imbalanced classification tasks. It provides a robust measure of a model's ability to distinguish between positive and negative instances, offering a **balanced assessment even when classes are disproportionate**. This allow us to gauge the model's discriminative power across different threshold settings, crucial in scenarios where correctly identifying positive instances (defaulters) is of vital importance."

We aimed at having a model with a ROC AUC of at least 0.80.

The next cells will present a bit of the workflow adopted in this stage of the project. All of the transformers applied to the dataframe can be seen in the folder feature_models.

In [ ]:
df_train = utils.load_df(file_path="data/df_train.csv")
df_valid = utils.load_df(file_path="data/df_validation.csv")
df_test = utils.load_df(file_path="data/df_test.csv")

df_previous = utils.load_df(file_path="data/home-credit-default-risk/previous_application.csv")
df_installments = utils.load_df(file_path="data/home-credit-default-risk/installments_payments.csv")
df_bureau = utils.load_df(file_path="data/home-credit-default-risk/bureau.csv")

The shape of the data is: (153755, 122)
load_df took 2.378 seconds

The shape of the data is: (61502, 122)
load_df took 1.086 seconds

The shape of the data is: (92254, 122)
load_df took 2.257 seconds

The shape of the data is: (1670214, 37)
load_df took 10.915 seconds

The shape of the data is: (13605401, 8)
load_df took 25.102 seconds

The shape of the data is: (1716428, 17)
load_df took 4.672 seconds

The shape of the data is: (32, 1)
load_df took 0.015 seconds



Some of the ideas behind the feature engineering process:

For the **previous application** we computed, for each client:
- the quantity of previous applications,
- the quantity of rejected/accepted/cancelled/unused_offer applications,
- aggregates for the continuous features for each client into "mean", "median", "max", "min".

In the **Installments dataframe**, we generated:
- the ratio of paid value by installment value (aggregates),
- flag if client ever paid less,
- delayed days of a payment (aggregates),
- flag if a client has delayed a payment.
- delayed days of a payment in the first year of application (365 days)

For the **bureau data** we will compute, for each client:
- aggregates for the continuous features for each client into "mean", "median", "max", "min".
- application counts,
- creating a credit active pivot table,
- financial ratios (median, max, and min values for each SK_ID_CURR).

In [ ]:
df_previous_transf = feat_eng.PreviousApplicationTransformer().fit_transform(df_previous)
df_installments_transf = feat_eng.InstallmentsTransformer().fit_transform(df_installments)
df_bureau_transf = feat_eng.BureauTransformer().fit_transform(df_bureau)

Our preprocessing and feature engineering pipelines were applied in the following step.

In [ ]:
def preprocess_and_drop_features(
    df, df_previous_transf, df_installments_transf, df_bureau_transf, feats_to_keep
):
    df_transformed = df.merge(df_previous_transf, on="SK_ID_CURR", how="left").merge(
        df_installments_transf, on="SK_ID_CURR", how="left"
    ).merge(
        df_bureau_transf, on="SK_ID_CURR", how="left"
    )

    X = df_transformed.drop(["TARGET", "SK_ID_CURR"], axis=1)
    y = df_transformed["TARGET"]

    feat_eng_pipe2 = Pipeline(
    steps=[
        ("drop_mode_avg", feat_eng.DropModeAVG()),
        ('zero_null_transformer', feat_eng.ZeroToNullTransformer(columns="YEARS_BEGINEXPLUATATION_MEDI")),
        ('multiply_by_neg1', feat_eng.MultiplyByNeg1(columns=[col for col in df.columns if col.startswith('DAYS_')])),
        ('days_empl_anomaly', feat_eng.HandleDaysEmployedAnomaly()),
        ('map_loan_titles', feat_eng.ApplyMapToOrganization(column="ORGANIZATION_TYPE", similarity_threshold=70)),
        ('ext_sources_transformer', feat_eng.ExternalSourcesTransformer()),
        ('financial_ratio_transformer', feat_eng.FinancialRatioTransformer()),
        ('age_employment_transformer', feat_eng.AgeAndEmploymentTransformer()),
        ]
    )

    X_FE = feat_eng_pipe2.fit_transform(X)
    num_feats, cat_feats, binary_feats, highcard_feats = utils.extract_features(X_FE)

    preprocess_pipe = pipelines.create_preprocess_pipeline(
        num_feats, binary_feats, highcard_feats
    )
    X_ready = preprocess_pipe.fit_transform(X_FE)
    X_ready = X_ready.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))
    
    return X_ready, y

In [ ]:
X_train, y_train = preprocess_and_drop_features(
    df=df_train,
    df_previous_transf=df_previous_transf,
    df_installments_transf=df_installments_transf,
    df_bureau_transf=df_bureau_transf
)
print(X_train.shape, y_train.size)

(153755, 32) 153755


,num__AMT_CREDIT,num__AMT_ANNUITY,num__DAYS_BIRTH,num__DAYS_EMPLOYED,num__OWN_CAR_AGE,num__REG_CITY_NOT_LIVE_CITY,num__EXT_SOURCE_1,num__EXT_SOURCE_2,num__EXT_SOURCE_3,num__PREV_name_contract_refused,...,num__BU_debt_credit_sum_ratio_max,num__EXT_SOURCES_prod,num__EXT_SOURCES_sum,num__EXT_SOURCES_mean,num__amt_credit_annuity_ratio,num__amt_goods_price_annuity_ratio,num__amt_goods_price_credit_ratio,num__amt_goods_price_children_ratio,num__days_employed_percent,cat__NAME_EDUCATION_TYPE_Higher_education
0,-0.618,-0.878,-0.969,-0.244,NaN,1.000,-1.233,-1.375,NaN,0.500,...,NaN,-0.754,-1.353,-1.848,0.000,0.056,0.642,-0.337,0.023,0.000
1,1.246,0.545,0.012,-0.179,0.300,1.000,0.061,0.152,NaN,NaN,...,-0.796,0.393,-0.013,0.204,1.229,0.742,-0.671,0.269,-0.256,1.000
2,1.918,0.885,-0.368,0.441,NaN,0.000,0.027,0.046,0.197,NaN,...,-0.758,-0.145,0.856,0.184,1.554,1.391,-0.125,0.721,0.542,1.000


Here we define our cross-validation for our models.

In [ ]:
def randomized_search_LGBM(X, y, param_distributions={"max_depth": randint(2, 10)}):
    lgbm_model = LGBMClassifier(random_state=1, class_weight='balanced', verbose=-1, learning_rate=0.01, n_estimators=10)
    randomized_search = RandomizedSearchCV(lgbm_model, param_distributions=param_distributions, n_iter=10, cv=5, random_state=1, scoring='roc_auc')
    
    return randomized_search

In [ ]:
randomized_search4 = randomized_search_LGBM(X_train, y_train)
randomized_search4 = randomized_search4.fit(X_train, y_train)
randomized_search4

RandomizedSearchCV(cv=5,
                   estimator=LGBMClassifier(class_weight='balanced',
                                            learning_rate=0.01, n_estimators=10,
                                            random_state=1, verbose=-1),
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002820186CF10>},
                   random_state=1, scoring='roc_auc')

In [ ]:
lgbm_results4 = {
    'model': randomized_search4.best_estimator_,
    'best_params': randomized_search4.best_params_,
    'best_score': randomized_search4.best_score_
}

lgbm_results4

{'model': LGBMClassifier(class_weight='balanced', learning_rate=0.01, max_depth=9,
                n_estimators=10, random_state=1, verbose=-1),
 'best_params': {'max_depth': 9},
 'best_score': 0.734603979468497}

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
best_lgbm_model4 = randomized_search4.best_estimator_
model_select.plot_feature_importance(best_lgbm_model4, X_train, y_train, ax=ax)

<img src="images/03_feature_importance.png" width=800>

In our latest round of feature engineering, despite several promising additions to our model, the **results remained somewhat discouraging**, yielding a **ROC AUC of 0.7343**. Notably, many of the newly **engineered features** secured positions **in the Top 10** in terms of importance. However, this apparent success did not translate into a substantial improvement in performance. To provide context, **our initial run** focused solely on the main datatable, producing a baseline **ROC AUC of 0.7186**.

<h1 style="color:#774CAD">Feature Selection</h1>

In [ ]:
print(f"Our data has currently {X_train.shape[1]} features")

Our data has currently 324 features


Our dataframe had at this stage many features, this increases the chances that our models will be focusing efforts on features that don't add much to our predictions and increases the dimensional space of our features (curse of dimensionality). Some of these features can be multicolinear as well.

The **following steps** were performed to reduce the amount of features:
- Remove features with more than 75% Null Values.
- Check and remove colinear features, but only if they were not deemed important contributors to the model's performance.
- Drop irrelevant features for the LGBM model.

As a result, only 37 features were maintained.

<h1 style="color:#774CAD">Model</h1>

In this section, we ran 4 models: **Decision Tree, XGBoost, LightGBM and CatBoost**. They were input in a cross-validation setup with 5 folds and randomized search for hyperparameters tuning.

In [ ]:
class_weights = compute_class_weight('balanced', classes=[0, 1], y=y_train)
class_weight = {0: class_weights[0], 1: class_weights[1]}

param_distributions = {
    'DecisionTree': {
        'model__max_depth': randint(3, 10),
        'model__min_samples_leaf': randint(30, 101)
    },
    'XGBoost': {
        'model__n_estimators': randint(10, 51),
        'model__learning_rate': uniform(0.01, 0.05),
        'model__max_depth': randint(3, 10)
    },
    'LGBM': {
        'model__n_estimators': randint(10, 51),
        'model__learning_rate': uniform(0.01, 0.05),
        'model__max_depth': randint(3, 10),
        'model__class_weight': [None, class_weight]
    },
    'CatBoost': {
        'model__iterations': randint(10, 51),
        'model__learning_rate': uniform(0.01, 0.05),
        'model__depth': randint(3, 10),
        'model__class_weights': [None, class_weight]
    },
}

models = [
    ('DecisionTree', DecisionTreeClassifier(random_state=1, class_weight='balanced')),
    ('XGBoost', XGBClassifier(random_state=1, class_weight=class_weight)), 
    ('LGBM', LGBMClassifier(random_state=1, class_weight='balanced', verbose=-1)),
    ('CatBoost', CatBoostClassifier(random_state=1, class_weights=class_weight, verbose=False)),
]


In [ ]:
results = {}
feat_importance_result = {}

for model_name, model in models:
    model_pipeline = Pipeline([
        ('model', model)
    ])

    param_dist = param_distributions.get(model_name, {})
    randomized_search = RandomizedSearchCV(model_pipeline, param_distributions=param_dist, n_iter=10, cv=5, scoring="roc_auc", random_state=1)
    randomized_search.fit(X_train, y_train)
        
    results[model_name] = {
        'model': randomized_search.best_estimator_,
        'best_params': randomized_search.best_params_,
        'best_score': randomized_search.best_score_,
    }



All of the models presented similar results in terms of ROC AUC.

<img src="images/04_roc-curve.png" width=400>

Through the confusion matrices bellow, we see however that the **XGBoost** did not perform well for the chosen threshold because it **mostly predicted values for the majority class** (0-Non-defaulters).

All of the **other models** seem to present **very similar results** of precision and recall for both classes, with the **Decision Tree performing a little better** in predicting correctly the class **1-Defauters**.

<img src="images/05_confusion-matrices.png" width=800>

We jump now into a analysis of the **Precision-Recall curve** to try to find an optimal threshold for our results (but still with focus on Recall for the defaulters class).

<img src="images/06_threshold_analysis.png" width=950>

We see here that finding the **optimal F1-Score comes with an obvious loss in Recall** for most of the models. This value drop from about 66% to 45%. In the context where we would like to predict defaulters, this change in threshold would not represent an advantage.

Something to notice here is that the curves of the XGBoost and CatBoost are higher than the other models, which depicts a slightly better precision-recall curve.

In the case of the **XGBoost**, the situation is completely different, with the standard threshold of 0.5 the levels of recall for our class of interest was 0. In this case, we applied a funciton to return a threshold for a given recall value.

The models were further evaluated in terms of feature importance and using the SHAP library. **Insights about the most relevant features can be seen in the conclusion**.

The models were lastly checked on the test data. Here, we checked still for all the boosting models, since the goal of the work was to deploy some machine learning models (not only one). The results were very similar to what was obtained using the validation set.

<h1 style="color:#774CAD">Deployment</h1>

The deployment of our machine learning models was made using FastAPI. A Docker image was created and deployed on the Google Cloud Platform.

We ran into a problem with one of the One-Hot-Encoded features at this stage and deployed the model on the processed features. The depolyed model can be accessed through the link: https://risk-retail-banks-nhk7sfh42a-oe.a.run.app/docs.

<h1 style="color:#774CAD">Feasability Analysis</h1>

This analysis was performed for our LightGBM model results. Only the test dataset was used, so one can expect a much lower sum of issued credit than in the initial analysis of the necessity of the model.

In [ ]:
money_df = utils.load_df(file_path="data/money_df.csv")
print(f"Sum of given credit on the test data: ${money_df['AMT_CREDIT'].sum():,.2f}")

Sum of given credit on the test data: $55,365,306,399.00


<img src="images/07_feasability.png" width=650>

The proportions highlight that our model is rejecting about 23.4% of applied credit by Non-defaulters, as we evaluate bellow, this value represents about 13 Billion dollars of non-issued credit. This is directly translated as money loss for our bank institution.

With our model, we will correctly not issue 4.7% of default credits ($2.6 Bi), but still provide about 2.9% ($1.6 Bi).

In [ ]:
value_lost_non_defaulters = money_df.loc[(money_df['y'] == 0) & (money_df['y_hat'] == 1), 'AMT_CREDIT'].sum()
value_gained_non_defaulters = money_df.loc[(money_df['y'] == 0) & (money_df['y_hat'] == 0), 'AMT_CREDIT'].sum()

value_lost_defaulters = money_df.loc[(money_df['y'] == 1) & (money_df['y_hat'] == 0), 'AMT_CREDIT'].sum()
value_not_lost_defaulters = money_df.loc[(money_df['y'] == 1) & (money_df['y_hat'] == 1), 'AMT_CREDIT'].sum()

print("Non-defaulters")
print(f"Issued Credit Non-Defaulters: ${value_gained_non_defaulters:,.2f}")
print(f"Value Lost by Rejecting Non-Defaulters: ${value_lost_non_defaulters:,.2f}\n")
print("Defaulters")
print(f"Issued Credit True Defaulters: ${value_lost_defaulters:,.2f}")
print(f"Value Gained by Rejecting True Defaulters: ${value_not_lost_defaulters:,.2f}\n")
print(f"Total money gained or not lost: ${value_gained_non_defaulters + value_not_lost_defaulters:,.2f}")
print(f"Total money lost: ${value_lost_non_defaulters + value_lost_defaulters:,.2f}")


Non-defaulters
Issued Credit Non-Defaulters: $38,217,772,917.00
Value Lost by Rejecting Non-Defaulters: $12,956,082,079.50

Defaulters
Issued Credit True Defaulters: $1,586,499,286.50
Value Gained by Rejecting True Defaulters: $2,604,952,116.00

Total money gained or not lost: $40,822,725,033.00
Total money lost: $14,542,581,366.00


The results are quite discouraging since we got low values of precision and recall.

This model could still be applied, if there is always a credit expert who will evaluate rejected applications and decide if they should really be rejected or not. Many ways to improve the model results are discussed in the next section.

<h1 style="color:#774CAD">Conclusion</h1>

Our machine learning boosting models achieve results ROC AUC values around 0.76, having for the choosen/given threshold Recall slightly above 65%.

After this evaluation, we know that our models are far from perfect and that there is a lot of room for improvement. Some conclusions on the most important features can be however made:
- **External sources play a key role** in a way that if a client has bad sources they can be already denied a loan;
- Another vital feature is the **ratio between the requested amount and the final credit amount** (in the previous application);
- The **Higher Education** tends to push clients towards being good payers;
- Another feature that seems to define good payers is **ratio between the Current debt and Current credit amount** (in the Credit Bureau);

How to improve the model results:
- Invest time on the inexplored dataframes;
- Bring more aggregated features from the previous application/installments to the model;
- Invest computational resources to generate features automatically (too much time invested on feature engineering);
- Compute interest rates;
- Create new aggregations for subsamples of the dataframes, i.e. NAME_PRODUCT_TYPE == "Approved"/"Cancelled", or CREDIT_ACTIVE == "bad debt"; 
- Try to impute values and check if there is increase in performance;

General improvements:
- To improve the workflow, some investiment can be done on the pipelines and creating mechanisms to integrate the data better;
- Correlations can be transformed into a function and put in EDA.py.
- Deployment of the initial features and passing through the pipelines.